In [1]:
import requests
import xml.etree.ElementTree as ET
import json
import math
import time
import sys

# 1. 발급받은 API 키를 여기에 입력하세요
service_key = "50hjvXuloV4qFNdrIUOglZZ6RGV7uq7pvpP0oxT+EV57bvEGnWfvqbjL939z/yfj9ta/H2Cn382mGmHpm4wmcw=="
url = "http://apis.data.go.kr/1471000/CsmtcsIngdCpntInfoService01/getCsmtcsIngdCpntInfoService01"
# 한 번의 호출로 최대한 많은 데이터를 가져오도록 설정
ROWS_PER_PAGE = 1000 
OUTPUT_FILENAME = "korea_standard_ingredients_api.json"

def fetch_all_ingredients():
    """공공 API를 반복 호출하여 전체 성분 데이터를 가져오고 파일로 저장합니다."""
    
    all_ingredients = []
    
    # 1. totalCount (총 데이터 건수) 확인 요청
    print("API 총 데이터 건수를 확인합니다...")
    params = {"serviceKey": service_key, "pageNo": 1, "numOfRows": 1}
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        root = ET.fromstring(response.content)
        total_count_tag = root.find('.//totalCount')
        
        if total_count_tag is None:
            print("❌ totalCount 정보를 찾을 수 없습니다. API 응답 형식 확인이 필요합니다.")
            return
            
        total_count = int(total_count_tag.text)
        total_pages = math.ceil(total_count / ROWS_PER_PAGE)
        
        print(f"✅ 총 {total_count}개의 성분 정보가 확인되었습니다. 총 {total_pages} 페이지를 호출합니다.")
        
    except Exception as e:
        print(f"❌ 초기 API 요청 중 오류 발생: {e}", file=sys.stderr)
        return

    # 2. 전체 페이지를 순회하며 데이터 수집
    for page_no in range(1, total_pages + 1):
        print(f"➡️ 데이터 수집 중: Page {page_no} / {total_pages}")
        
        params = {
            "serviceKey": service_key, 
            "pageNo": page_no, 
            "numOfRows": ROWS_PER_PAGE
        }
        
        try:
            response = requests.get(url, params=params, timeout=20)
            response.raise_for_status()
            root = ET.fromstring(response.content)
            
            items = root.findall('.//item')
            
            for item in items:
                # 필요한 필드 추출
                ingr_kor_name = item.find('INGR_KOR_NAME').text if item.find('INGR_KOR_NAME') is not None else None
                ingr_eng_name = item.find('INGR_ENG_NAME').text if item.find('INGR_ENG_NAME') is not None else None
                cas_no = item.find('CAS_NO').text if item.find('CAS_NO') is not None else None
                
                # 유효한 한글 성분명이 있는 경우에만 저장
                if ingr_kor_name:
                    all_ingredients.append({
                        "원료명": ingr_kor_name,
                        "영문명": ingr_eng_name,
                        "CAS_NO": cas_no
                    })
                    
            # API 호출 간 부하 방지 및 안정성 확보를 위해 잠시 대기
            time.sleep(0.5) 

        except Exception as e:
            print(f"❌ Page {page_no} 처리 중 오류 발생: {e}", file=sys.stderr)
            time.sleep(5) # 오류 발생 시 더 오래 대기 후 재시도 (혹은 다음 페이지로)
            continue
            
    # 3. 수집된 데이터를 JSON 파일로 저장
    if all_ingredients:
        with open(OUTPUT_FILENAME, 'w', encoding='utf-8') as f:
            json.dump(all_ingredients, f, ensure_ascii=False, indent=4)
        
        print(f"\n🎉 전체 성분 데이터 수집 및 저장 완료!")
        print(f"📦 총 {len(all_ingredients)}개의 성분이 '{OUTPUT_FILENAME}'에 저장되었습니다.")
        
        # 성분 정제에 사용할 표준 한글 성분명 리스트 반환
        return [item['원료명'] for item in all_ingredients]
    else:
        print("❌ 수집된 성분 데이터가 없습니다.")
        return []

# 함수 실행 및 표준 성분 리스트 얻기
standard_ingredients_list = fetch_all_ingredients()

API 총 데이터 건수를 확인합니다...


❌ 초기 API 요청 중 오류 발생: 500 Server Error: Internal Server Error for url: http://apis.data.go.kr/1471000/CsmtcsIngdCpntInfoService01/getCsmtcsIngdCpntInfoService01?serviceKey=50hjvXuloV4qFNdrIUOglZZ6RGV7uq7pvpP0oxT%2BEV57bvEGnWfvqbjL939z%2Fyfj9ta%2FH2Cn382mGmHpm4wmcw%3D%3D&pageNo=1&numOfRows=1


In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import urljoin, unquote

# ----------------------------------------------------------------------
# 1단계: 모든 상세 URL 수집 함수
# ----------------------------------------------------------------------
def scrape_all_ingredient_urls(driver, max_page=1288):
    """
    페이지네이션을 이용하여 모든 성분 상세 페이지의 URL을 수집합니다.
    """
    BASE_URL = "https://coos.kr"
    all_detail_urls = set()
    
    # ⚠️ 실제 전체 스크래핑 시에는 max_page=1288로 설정해야 합니다.
    # 테스트를 위해 작은 값으로 설정합니다.
    # max_page = 3  # 테스트용
    
    print(f"==================================================")
    print(f"🔎 1단계: 총 {max_page} 페이지를 탐색하여 성분 URL을 수집합니다...")
    print(f"==================================================")
    
    for page_num in range(1, max_page + 1):
        list_url = f"{BASE_URL}/ingredients?page={page_num}"
        
        try:
            driver.get(list_url)
            time.sleep(1.5) # 로딩 대기 시간 (환경에 따라 조정 필요)
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # 성분 상세 URL은 '/ingredients/'로 시작하고 Query String(?)이 없는 <a> 태그의 href를 이용합니다.
            link_tags = soup.find_all(
                'a', 
                href=lambda href: href and href.startswith('/ingredients/') and '?' not in href
            )
            
            # 수집 및 중복 제거
            for tag in link_tags:
                url_path = tag['href']
                full_url = urljoin(BASE_URL, url_path)
                all_detail_urls.add(full_url)
            
            if page_num % 100 == 0 or page_num == 1 or page_num == max_page:
                print(f"   탐색 중: Page {page_num} / {max_page} | 현재 URL 수: {len(all_detail_urls)}개")

        except Exception as e:
            print(f"⚠️ Page {page_num} 로딩 중 오류 발생 ({e}). 건너뛰고 다음 페이지로 이동합니다.")
            continue
            
    print(f"✅ 1단계 완료: 총 {len(all_detail_urls)}개의 성분 상세 URL 수집 완료.")
    return list(all_detail_urls)

# ----------------------------------------------------------------------
# 2단계: 상세 페이지 정보 추출 함수
# ----------------------------------------------------------------------
def scrape_ingredient_details(driver, ingredient_url):
    """
    성분 상세 페이지에서 한글명, 영문명, CAS No.를 안정적으로 추출합니다.
    (동적 CSS 클래스 대신, 태그와 텍스트 레이블 기반)
    """
    driver.get(ingredient_url)
    time.sleep(1) # 로딩 대기
    
    details = {
        '원료명': unquote(ingredient_url.split('/')[-1]), # URL에서 임시로 이름 추출
        '영문명': None,
        'CAS_No': None,
        '설명_요약': None,
        '태그_목록': []
    }
    
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 1. 원료명 (한글) 추출: <h1> 태그 사용
        h1_tag = soup.find('h1')
        if h1_tag:
            details['원료명'] = h1_tag.text.strip()
            
            # 2. 영문명 추출: <h1> 태그 내부에 작은 div로 분리되어 있는 경우
            # <div style="...;">...</div> 형태를 찾습니다.
            english_name_div = h1_tag.find_next_sibling('div', style=lambda s: s and 'font-size' in s)
            if english_name_div:
                 details['영문명'] = english_name_div.text.strip()
                 
        # 3. CAS No. 및 상세 정보 추출: <dt> (레이블) / <dd> (값) 쌍 검색
        info_items = soup.find_all(['dt', 'dd'])
        for i in range(len(info_items) - 1):
            if info_items[i].name == 'dt':
                label = info_items[i].text.strip()
                value_tag = info_items[i+1]
                value = value_tag.text.strip() if value_tag and value_tag.name == 'dd' else None
                
                if value and value.lower() != 'none':
                    if 'INCI Name' in label:
                        # 이미 1번에서 추출했으나, 다시 확인하는 용도
                        if not details['영문명']:
                            details['영문명'] = value
                    elif 'CAS No' in label:
                        details['CAS_No'] = value

        # 4. 설명 요약 추출: 페이지 내 특정 CSS 클래스를 가진 div에서 추출 (이전 분석 참고)
        # 클래스가 변경될 수 있으므로, 내용이 긴 첫 번째 단락을 찾는 것도 대안
        summary_div = soup.find('div', class_=lambda c: c and 'css-m3enaf' in c)
        if summary_div:
            details['설명_요약'] = summary_div.text.strip()
            
        # 5. 태그/칩 목록 추출: 모든 칩 레이블 텍스트 수집
        chip_labels = soup.find_all('span', class_=lambda c: c and 'MuiChip-label' in c)
        details['태그_목록'] = [label.text.strip() for label in chip_labels if label.text.strip()]
        
    except Exception as e:
        print(f"   오류 발생: {ingredient_url} 상세 정보 추출 실패: {e}")
        
    return details

# ----------------------------------------------------------------------
# 메인 실행 블록
# ----------------------------------------------------------------------
if __name__ == "__main__":
    # --- 설정 ---
    # ⚠️ (필수) max_page 값을 '1288'로 설정해야 모든 페이지를 스크래핑합니다.
    MAX_PAGE = 3 # 테스트 실행을 위해 3페이지로 설정합니다.
    OUTPUT_FILENAME = "coos_ingredient_database.csv"
    
    # 1. 웹 드라이버 설정 (Selenium Manager가 자동으로 드라이버를 찾아 사용합니다.)
    print("🚀 웹 드라이버 초기화 중...")
    try:
        driver = webdriver.Chrome()
    except Exception as e:
        print(f"❌ 드라이버 초기화 실패. Chrome 설치 및 Selenium 버전 확인이 필요합니다: {e}")
        exit()
        
    start_time = time.time()
    
    # 2. 1단계 실행: 모든 상세 URL 수집
    all_urls = scrape_all_ingredient_urls(driver, max_page=MAX_PAGE)
    
    # 3. 2단계 실행: 상세 정보 추출
    print(f"\n==================================================")
    print(f"🔬 2단계: 수집된 {len(all_urls)}개의 URL에서 상세 정보를 추출합니다...")
    print(f"==================================================")
    
    final_data = []
    
    for i, url in enumerate(all_urls):
        data = scrape_ingredient_details(driver, url)
        data['상세_URL'] = url
        final_data.append(data)
        
        if (i + 1) % 5 == 0 or (i + 1) == len(all_urls):
            print(f"   진행 상황: {i + 1} / {len(all_urls)} ({((i + 1) / len(all_urls) * 100):.1f}%) 추출 완료")

    # 4. 드라이버 종료
    driver.quit()
    
    end_time = time.time()
    
    # 5. 데이터 저장 및 정리
    df = pd.DataFrame(final_data)
    
    # 리스트 형태의 태그 목록을 문자열로 변환하여 CSV에 저장하기 용이하게 합니다.
    df['태그_목록'] = df['태그_목록'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    
    df.to_csv(OUTPUT_FILENAME, index=False, encoding='utf-8-sig')
    
    print(f"\n==================================================")
    print(f"🎉 작업 완료!")
    print(f"총 소요 시간: {end_time - start_time:.2f}초")
    print(f"최종 데이터 파일: {OUTPUT_FILENAME}")
    print(f"총 {len(df)}개 성분 저장 완료.")
    print(f"==================================================")

🚀 웹 드라이버 초기화 중...
🔎 1단계: 총 3 페이지를 탐색하여 성분 URL을 수집합니다...
   탐색 중: Page 1 / 3 | 현재 URL 수: 20개
   탐색 중: Page 3 / 3 | 현재 URL 수: 60개
✅ 1단계 완료: 총 60개의 성분 상세 URL 수집 완료.

🔬 2단계: 수집된 60개의 URL에서 상세 정보를 추출합니다...
   진행 상황: 5 / 60 (8.3%) 추출 완료
   진행 상황: 10 / 60 (16.7%) 추출 완료
   진행 상황: 15 / 60 (25.0%) 추출 완료
   진행 상황: 20 / 60 (33.3%) 추출 완료
   진행 상황: 25 / 60 (41.7%) 추출 완료
   진행 상황: 30 / 60 (50.0%) 추출 완료
   진행 상황: 35 / 60 (58.3%) 추출 완료
   진행 상황: 40 / 60 (66.7%) 추출 완료
   진행 상황: 45 / 60 (75.0%) 추출 완료
   진행 상황: 50 / 60 (83.3%) 추출 완료
   진행 상황: 55 / 60 (91.7%) 추출 완료
   진행 상황: 60 / 60 (100.0%) 추출 완료

🎉 작업 완료!
총 소요 시간: 100.56초
최종 데이터 파일: coos_ingredient_database.csv
총 60개 성분 저장 완료.


In [5]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import urljoin, unquote

# ----------------------------------------------------------------------
# 1단계: 모든 상세 URL 수집 함수
# ----------------------------------------------------------------------
def scrape_all_ingredient_urls(driver, max_page):
    """
    페이지네이션을 이용하여 모든 성분 상세 페이지의 URL을 수집합니다.
    """
    BASE_URL = "https://coos.kr"
    all_detail_urls = set()
    
    print(f"==================================================")
    print(f"🔎 1단계: 총 {max_page} 페이지를 탐색하여 성분 URL을 수집합니다...")
    print(f"==================================================")
    
    for page_num in range(1, max_page + 1):
        list_url = f"{BASE_URL}/ingredients?page={page_num}"
        
        try:
            driver.get(list_url)
            # 로딩 대기 시간: 환경에 따라 조정이 필요하며, 너무 짧으면 데이터 누락, 너무 길면 시간 낭비
            time.sleep(1.5) 
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # 성분 상세 URL은 '/ingredients/'로 시작하고 Query String(?)이 없는 <a> 태그의 href를 이용
            link_tags = soup.find_all(
                'a', 
                href=lambda href: href and href.startswith('/ingredients/') and '?' not in href
            )
            
            # 수집 및 중복 제거
            for tag in link_tags:
                url_path = tag['href']
                full_url = urljoin(BASE_URL, url_path)
                all_detail_urls.add(full_url)
            
            if page_num % 100 == 0 or page_num == 1 or page_num == max_page:
                print(f"   탐색 중: Page {page_num} / {max_page} | 현재 URL 수: {len(all_detail_urls)}개")

        except Exception as e:
            print(f"⚠️ Page {page_num} 로딩 중 오류 발생 ({e}). 건너뛰고 다음 페이지로 이동합니다.")
            # 오류가 나도 프로그램을 멈추지 않고 다음 페이지로 계속 진행
            continue
            
    print(f"✅ 1단계 완료: 총 {len(all_detail_urls)}개의 성분 상세 URL 수집 완료.")
    return list(all_detail_urls)

# ----------------------------------------------------------------------
# 2단계: 상세 페이지 정보 추출 함수 (태그 노이즈 필터링 적용)
# ----------------------------------------------------------------------
def scrape_ingredient_details(driver, ingredient_url):
    """
    성분 상세 페이지에서 한글명, 영문명, CAS No. 및 태그 정보를 추출합니다.
    (태그 목록에서 불필요한 노이즈 코드 제거 로직 포함)
    """
    driver.get(ingredient_url)
    time.sleep(1) # 로딩 대기
    
    details = {
        '원료명': unquote(ingredient_url.split('/')[-1]), 
        '영문명': None,
        'CAS_No': None,
        '설명_요약': None,
        '태그_목록': []
    }
    
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 1. 원료명 (한글) 추출: <h1> 태그
        h1_tag = soup.find('h1')
        if h1_tag:
            details['원료명'] = h1_tag.text.strip()
            
            # 2. 영문명 추출: <h1> 바로 다음의 div
            english_name_div = h1_tag.find_next_sibling('div', style=lambda s: s and 'font-size' in s)
            if english_name_div:
                 details['영문명'] = english_name_div.text.strip()
                 
        # 3. CAS No. 및 상세 정보 추출: <dt> (레이블) / <dd> (값) 쌍 검색
        info_items = soup.find_all(['dt', 'dd'])
        for i in range(len(info_items) - 1):
            if info_items[i].name == 'dt':
                label = info_items[i].text.strip()
                value_tag = info_items[i+1]
                value = value_tag.text.strip() if value_tag and value_tag.name == 'dd' else None
                
                if value and value.lower() != 'none':
                    if 'INCI Name' in label and not details['영문명']:
                        details['영문명'] = value
                    elif 'CAS No' in label:
                        details['CAS_No'] = value

        # 4. 설명 요약 추출: CSS 클래스 기반
        summary_div = soup.find('div', class_=lambda c: c and 'css-m3enaf' in c)
        if summary_div:
            details['설명_요약'] = summary_div.text.strip()
            
        # 5. 태그/칩 목록 추출 및 필터링
        chip_labels = soup.find_all('span', class_=lambda c: c and 'MuiChip-label' in c)
        
        # 🚫 필터링할 불필요한 노이즈 태그 목록 정의
        # 짧은 길이, 국가/언어 코드, 불명확한 상태 코드 제거
        NOISE_TAGS = {
            'KO', 'EN', 'JP', 'AI', 'EU', '식', '가능', '불가', 'AI,', 'EU,', 
            '식,', '가능,', '불가,', 'INCI Name', 'CAS No' # 레이블이 태그로 잘못 들어오는 경우 방지
        } 
        
        extracted_tags = []
        for label in chip_labels:
            tag_text = label.text.strip()
            # 텍스트가 비어있지 않고, 노이즈 목록에 없으며, 길이가 2자 초과인 경우만 추출
            if tag_text and tag_text not in NOISE_TAGS and len(tag_text) > 2: 
                extracted_tags.append(tag_text)
                
        details['태그_목록'] = extracted_tags
        
    except Exception as e:
        print(f"   오류 발생: {ingredient_url} 상세 정보 추출 실패: {e}")
        
    return details

# ----------------------------------------------------------------------
# 메인 실행 블록
# ----------------------------------------------------------------------
if __name__ == "__main__":
    # --- 설정 ---
    # ⚠️ 전체 데이터 수집을 위한 최종 설정입니다.
    MAX_PAGE = 1288 
    OUTPUT_FILENAME = "coos_ingredient_database_final.csv"
    
    # 1. 웹 드라이버 설정 (Selenium Manager가 자동으로 드라이버를 찾아 사용합니다.)
    print("🚀 웹 드라이버 초기화 중...")
    try:
        driver = webdriver.Chrome()
    except Exception as e:
        print(f"❌ 드라이버 초기화 실패. Chrome 설치 및 Selenium 버전 확인이 필요합니다: {e}")
        exit()
        
    start_time = time.time()
    
    # 2. 1단계 실행: 모든 상세 URL 수집
    # max_page를 인자로 전달합니다.
    all_urls = scrape_all_ingredient_urls(driver, max_page=MAX_PAGE)
    
    # 3. 2단계 실행: 상세 정보 추출
    print(f"\n==================================================")
    print(f"🔬 2단계: 수집된 {len(all_urls)}개의 URL에서 상세 정보를 추출합니다...")
    print(f"==================================================")
    
    final_data = []
    
    for i, url in enumerate(all_urls):
        data = scrape_ingredient_details(driver, url)
        data['상세_URL'] = url
        final_data.append(data)
        
        # 진행 상황 출력
        if (i + 1) % 50 == 0 or (i + 1) == len(all_urls) or i == 0:
            print(f"   진행 상황: {i + 1} / {len(all_urls)} ({((i + 1) / len(all_urls) * 100):.1f}%) 추출 완료")

    # 4. 드라이버 종료
    driver.quit()
    
    end_time = time.time()
    
    # 5. 데이터 저장 및 정리
    df = pd.DataFrame(final_data)
    
    # 리스트 형태의 태그 목록을 문자열로 변환하여 CSV에 저장
    df['태그_목록'] = df['태그_목록'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    
    df.to_csv(OUTPUT_FILENAME, index=False, encoding='utf-8-sig')
    
    print(f"\n==================================================")
    print(f"🎉 작업 완료!")
    print(f"총 소요 시간: {end_time - start_time:.2f}초")
    print(f"최종 데이터 파일: {OUTPUT_FILENAME}")
    print(f"총 {len(df)}개 성분 저장 완료.")
    print(f"==================================================")


🚀 웹 드라이버 초기화 중...
🔎 1단계: 총 1288 페이지를 탐색하여 성분 URL을 수집합니다...
   탐색 중: Page 1 / 1288 | 현재 URL 수: 20개
   탐색 중: Page 100 / 1288 | 현재 URL 수: 2000개
   탐색 중: Page 200 / 1288 | 현재 URL 수: 3998개
   탐색 중: Page 300 / 1288 | 현재 URL 수: 5998개
   탐색 중: Page 400 / 1288 | 현재 URL 수: 7998개
   탐색 중: Page 500 / 1288 | 현재 URL 수: 9998개
   탐색 중: Page 600 / 1288 | 현재 URL 수: 11998개
   탐색 중: Page 700 / 1288 | 현재 URL 수: 13998개
   탐색 중: Page 800 / 1288 | 현재 URL 수: 15998개
   탐색 중: Page 900 / 1288 | 현재 URL 수: 17998개
   탐색 중: Page 1000 / 1288 | 현재 URL 수: 19998개
   탐색 중: Page 1100 / 1288 | 현재 URL 수: 21998개
   탐색 중: Page 1200 / 1288 | 현재 URL 수: 23998개
   탐색 중: Page 1288 / 1288 | 현재 URL 수: 25750개
✅ 1단계 완료: 총 25750개의 성분 상세 URL 수집 완료.

🔬 2단계: 수집된 25750개의 URL에서 상세 정보를 추출합니다...
   진행 상황: 1 / 25750 (0.0%) 추출 완료
   진행 상황: 50 / 25750 (0.2%) 추출 완료
   진행 상황: 100 / 25750 (0.4%) 추출 완료
   진행 상황: 150 / 25750 (0.6%) 추출 완료
   진행 상황: 200 / 25750 (0.8%) 추출 완료
   진행 상황: 250 / 25750 (1.0%) 추출 완료
   진행 상황: 300 / 25750 (1.2%) 추출 완료
   진행 상황: 

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=64907): Read timed out. (read timeout=120)

In [7]:
import time
import pandas as pd
import random # 🌟 무작위 대기 시간 적용을 위해 추가
import os # 🌟 파일 존재 여부 확인을 위해 추가
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import urljoin, unquote

# ----------------------------------------------------------------------
# 1단계: 모든 상세 URL 수집 함수 (지연 시간 강화 및 페이지 재개 기능 포함)
# ----------------------------------------------------------------------
def scrape_all_ingredient_urls(driver, max_page, url_file, last_page_file):
    """
    페이지네이션을 이용하여 모든 성분 상세 페이지의 URL을 수집합니다.
    """
    BASE_URL = "https://coos.kr"
    all_detail_urls = set()
    
    # 🌟 1-1. 기존 URL 파일에서 URL 목록 로드 (만약 1단계가 완전히 완료되지 않았을 경우)
    if os.path.exists(url_file):
        try:
            with open(url_file, 'r', encoding='utf-8') as f:
                loaded_urls = {line.strip() for line in f if line.strip()}
            all_detail_urls.update(loaded_urls)
            print(f"   -> 기존 URL 파일에서 {len(all_detail_urls)}개의 URL을 로드했습니다.")
        except:
             print("   -> 기존 URL 파일 로드에 실패했습니다. 처음부터 URL을 수집합니다.")

    # 🌟 1-2. 마지막 성공 페이지 번호 로드
    start_page = 1
    if os.path.exists(last_page_file):
        try:
            with open(last_page_file, 'r') as f:
                start_page = int(f.read().strip())
                # 이미 처리된 페이지는 건너뜁니다.
                print(f"   ✅ 1단계 재개: {start_page} 페이지부터 탐색을 시작합니다.")
        except:
            pass # 파일이 손상되었다면 1부터 시작

    print(f"==================================================")
    print(f"🔎 1단계: 총 {max_page} 페이지 중 {start_page} 페이지부터 탐색을 시작합니다...")
    print(f"==================================================")
    
    for page_num in range(start_page, max_page + 1):
        list_url = f"{BASE_URL}/ingredients?page={page_num}"
        
        try:
            driver.get(list_url)
            # 🌟 대기 시간 강화: 1.5초 ~ 3.5초 사이의 무작위 대기
            time.sleep(random.uniform(1.5, 3.5)) 
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            link_tags = soup.find_all(
                'a', 
                href=lambda href: href and href.startswith('/ingredients/') and '?' not in href
            )
            
            # 수집 및 중복 제거
            new_urls = []
            for tag in link_tags:
                url_path = tag['href']
                full_url = urljoin(BASE_URL, url_path)
                if full_url not in all_detail_urls:
                    all_detail_urls.add(full_url)
                    new_urls.append(full_url)
            
            # 🌟 페이지 성공 시 현재 페이지 번호 저장 및 수집된 URL 목록 추가 저장
            with open(last_page_file, 'w') as f:
                f.write(str(page_num + 1)) # 다음 시작 페이지를 저장

            with open(url_file, 'a', encoding='utf-8') as f:
                for url in new_urls:
                    f.write(f"{url}\n")
            
            # 🌟 매 100페이지마다 더 긴 휴식 시간을 가집니다. (서버 부하 감소)
            if page_num % 100 == 0:
                print(f"   === 100페이지 휴식 시작 (10초) ===")
                time.sleep(10)
                
            if page_num % 50 == 0 or page_num == max_page:
                print(f"   탐색 완료: Page {page_num} / {max_page} | 현재 URL 수: {len(all_detail_urls)}개")

        except Exception as e:
            print(f"⚠️ Page {page_num} 로딩 중 오류 발생 ({e}). {last_page_file}에 저장 후 종료 대기...")
            time.sleep(5) # 오류 발생 시 추가 대기 후 재시도
            # 오류 발생 시 마지막 성공 페이지가 저장되어 있으므로, 프로그램이 종료되어도 재개 가능
            raise # 오류 발생 시 스크립트를 중단하여 재실행을 유도

    # 🌟 모든 페이지 탐색이 성공적으로 완료되면 페이지 추적 파일을 삭제합니다.
    if os.path.exists(last_page_file):
        os.remove(last_page_file)
            
    print(f"✅ 1단계 완료: 총 {len(all_detail_urls)}개의 성분 상세 URL 수집 완료.")
    return list(all_detail_urls)

# ----------------------------------------------------------------------
# 2단계: 상세 페이지 정보 추출 함수 (태그 노이즈 필터링 적용)
# ----------------------------------------------------------------------
def scrape_ingredient_details(driver, ingredient_url):
    driver.get(ingredient_url)
    # 🌟 대기 시간 강화: 1초 ~ 2.5초 사이의 무작위 대기
    time.sleep(random.uniform(1.0, 2.5)) 
    
    details = {
        '원료명': unquote(ingredient_url.split('/')[-1]), 
        '영문명': None,
        'CAS_No': None,
        '설명_요약': None,
        '태그_목록': []
    }
    
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 1. 원료명 (한글) 추출: <h1> 태그
        h1_tag = soup.find('h1')
        if h1_tag:
            details['원료명'] = h1_tag.text.strip()
            
            # 2. 영문명 추출: <h1> 바로 다음의 div
            english_name_div = h1_tag.find_next_sibling('div', style=lambda s: s and 'font-size' in s)
            if english_name_div:
                 details['영문명'] = english_name_div.text.strip()
                 
        # 3. CAS No. 및 상세 정보 추출: <dt> (레이블) / <dd> (값) 쌍 검색
        info_items = soup.find_all(['dt', 'dd'])
        for i in range(len(info_items) - 1):
            if info_items[i].name == 'dt':
                label = info_items[i].text.strip()
                value_tag = info_items[i+1]
                value = value_tag.text.strip() if value_tag and value_tag.name == 'dd' else None
                
                if value and value.lower() != 'none':
                    if 'INCI Name' in label and not details['영문명']:
                        details['영문명'] = value
                    elif 'CAS No' in label:
                        details['CAS_No'] = value

        # 4. 설명 요약 추출: CSS 클래스 기반
        summary_div = soup.find('div', class_=lambda c: c and 'css-m3enaf' in c)
        if summary_div:
            details['설명_요약'] = summary_div.text.strip()
            
        # 5. 태그/칩 목록 추출 및 필터링
        chip_labels = soup.find_all('span', class_=lambda c: c and 'MuiChip-label' in c)
        NOISE_TAGS = {
            'KO', 'EN', 'JP', 'AI', 'EU', '식', '가능', '불가', 'AI,', 'EU,', 
            '식,', '가능,', '불가,', 'INCI Name', 'CAS No' 
        } 
        
        extracted_tags = []
        for label in chip_labels:
            tag_text = label.text.strip()
            if tag_text and tag_text not in NOISE_TAGS and len(tag_text) > 2: 
                extracted_tags.append(tag_text)
                
        details['태그_목록'] = extracted_tags
        
    except Exception as e:
        print(f"   오류 발생: {ingredient_url} 상세 정보 추출 실패: {e}")
        time.sleep(3) # 상세 페이지 오류 시 추가 대기
        
    return details

# ----------------------------------------------------------------------
# 메인 실행 블록
# ----------------------------------------------------------------------
if __name__ == "__main__":
    # --- 설정 ---
    MAX_PAGE = 1288 
    OUTPUT_FILENAME = "coos_ingredient_database.csv"
    # 🌟 1단계 URL 수집 재개를 위한 파일 추가
    URL_LIST_FILE = "coos_all_urls.txt"
    LAST_PAGE_FILE = "last_scraped_page.txt"
    
    # 1. 웹 드라이버 설정
    print("🚀 웹 드라이버 초기화 중...")
    try:
        driver = webdriver.Chrome()
    except Exception as e:
        print(f"❌ 드라이버 초기화 실패. Chrome 설치 및 Selenium 버전 확인이 필요합니다: {e}")
        exit()
        
    start_time = time.time()
    
    # 2. 1단계 실행: 모든 상세 URL 수집 (페이지 재개 기능 사용)
    try:
        all_urls = scrape_all_ingredient_urls(driver, max_page=MAX_PAGE, url_file=URL_LIST_FILE, last_page_file=LAST_PAGE_FILE)
    except Exception as e:
        # 1단계에서 오류가 나면 드라이버만 닫고 종료하여 재실행을 준비합니다.
        print(f"\n❌ 1단계 URL 수집 중 치명적인 오류 발생: {e}. 다음 재실행 시 {LAST_PAGE_FILE}부터 재개됩니다.")
        driver.quit()
        exit()
    
    # 🌟 3. 재개 로직 적용 및 URL 목록 필터링 (2단계)
    scraped_urls = set()
    existing_data = []

    if os.path.exists(OUTPUT_FILENAME):
        print(f"\n✅ 기존 상세 데이터 파일 '{OUTPUT_FILENAME}' 발견. 2단계 재개를 시도합니다.")
        try:
            df_existing = pd.read_csv(OUTPUT_FILENAME, encoding='utf-8-sig')
            scraped_urls = set(df_existing['상세_URL'].dropna())
            existing_data = df_existing.to_dict('records')
            
            existing_data = [d for d in existing_data if d.get('상세_URL')] 
            print(f"   -> 이미 {len(scraped_urls)}개의 성분 데이터가 수집되어 있습니다. 미수집 URL만 처리합니다.")
        except Exception as e:
             print(f"   -> 기존 파일을 읽는 중 오류 발생 ({e}). 처음부터 2단계 수집을 시작합니다.")

    # 미수집된 URL만 남깁니다.
    pending_urls = [url for url in all_urls if url not in scraped_urls]
    
    print(f"\n==================================================")
    print(f"🔬 2단계: 미처리된 {len(pending_urls)}개의 URL에서 상세 정보를 추출합니다...")
    print(f"==================================================")
    
    final_data = existing_data # 기존 데이터 + 새로 수집할 데이터

    for i, url in enumerate(pending_urls):
        
        data = scrape_ingredient_details(driver, url)
        data['상세_URL'] = url
        final_data.append(data)
        
        # 🌟 매 100개마다 중간 저장 (데이터 유실 방지)
        if (len(final_data) - len(existing_data)) % 100 == 0 and (len(final_data) - len(existing_data)) > 0:
            df_temp = pd.DataFrame(final_data)
            df_temp['태그_목록'] = df_temp['태그_목록'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
            df_temp.to_csv(OUTPUT_FILENAME, index=False, encoding='utf-8-sig')
            print(f"   [중간 저장] 현재까지 총 {len(final_data)}개 성분 저장 완료.")

        current_processed = i + 1
        total_pending = len(pending_urls)
        
        if current_processed % 50 == 0 or current_processed == total_pending or current_processed == 1:
            total_scraped_now = len(existing_data) + current_processed
            print(f"   진행 상황: {current_processed} / {total_pending} 미처리 항목 완료 | 총 {total_scraped_now}개 ({total_scraped_now / len(all_urls) * 100:.1f}%)")


    # 4. 드라이버 종료
    driver.quit()
    
    end_time = time.time()
    
    # 5. 최종 데이터 저장 및 정리
    df = pd.DataFrame(final_data)
    
    # 리스트 형태의 태그 목록을 문자열로 변환하여 CSV에 저장
    df['태그_목록'] = df['태그_목록'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    
    df.to_csv(OUTPUT_FILENAME, index=False, encoding='utf-8-sig')
    
    print(f"\n==================================================")
    print(f"🎉 작업 완료!")
    print(f"총 소요 시간: {end_time - start_time:.2f}초")
    print(f"최종 데이터 파일: {OUTPUT_FILENAME}")
    print(f"총 {len(df)}개 성분 저장 완료.")
    print(f"==================================================")


🚀 웹 드라이버 초기화 중...
   -> 기존 URL 파일에서 300개의 URL을 로드했습니다.
   ✅ 1단계 재개: 16 페이지부터 탐색을 시작합니다.
🔎 1단계: 총 1288 페이지 중 16 페이지부터 탐색을 시작합니다...
   탐색 완료: Page 50 / 1288 | 현재 URL 수: 1000개
   === 100페이지 휴식 시작 (10초) ===
   탐색 완료: Page 100 / 1288 | 현재 URL 수: 2000개
   탐색 완료: Page 150 / 1288 | 현재 URL 수: 3000개
   === 100페이지 휴식 시작 (10초) ===
   탐색 완료: Page 200 / 1288 | 현재 URL 수: 3998개
   탐색 완료: Page 250 / 1288 | 현재 URL 수: 4998개
   === 100페이지 휴식 시작 (10초) ===
   탐색 완료: Page 300 / 1288 | 현재 URL 수: 5998개
   탐색 완료: Page 350 / 1288 | 현재 URL 수: 6998개
   === 100페이지 휴식 시작 (10초) ===
   탐색 완료: Page 400 / 1288 | 현재 URL 수: 7998개
   탐색 완료: Page 450 / 1288 | 현재 URL 수: 8998개
   === 100페이지 휴식 시작 (10초) ===
   탐색 완료: Page 500 / 1288 | 현재 URL 수: 9998개
   탐색 완료: Page 550 / 1288 | 현재 URL 수: 10998개
   === 100페이지 휴식 시작 (10초) ===
   탐색 완료: Page 600 / 1288 | 현재 URL 수: 11998개
   탐색 완료: Page 650 / 1288 | 현재 URL 수: 12998개
   === 100페이지 휴식 시작 (10초) ===
   탐색 완료: Page 700 / 1288 | 현재 URL 수: 13998개
   탐색 완료: Page 750 / 1288 | 현재 URL 수: 149

WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=141.0.7390.66)
Stacktrace:
0   chromedriver                        0x000000010087f5f0 cxxbridge1$str$ptr + 2894960
1   chromedriver                        0x000000010087752c cxxbridge1$str$ptr + 2861996
2   chromedriver                        0x000000010039d5ec _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74324
3   chromedriver                        0x00000001003956b0 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 41752
4   chromedriver                        0x0000000100388220 cxxbridge1$string$len + 4708
5   chromedriver                        0x0000000100389cb8 cxxbridge1$string$len + 11516
6   chromedriver                        0x0000000100388698 cxxbridge1$string$len + 5852
7   chromedriver                        0x0000000100387f88 cxxbridge1$string$len + 4044
8   chromedriver                        0x0000000100387d14 cxxbridge1$string$le